<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [1]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/mhdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mhdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mhdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [3]:
class Preprocessor:

    def __init__(self, stopwords):
        self.stopwords = stopwords
        self.ps = PorterStemmer()
        self.lem = WordNetLemmatizer()


    def preprocess(self, text, is_query=False):
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        tokens = self.word_tokenize(text)
        tokens = self.normalize(tokens)
        if not is_query:
            tokens = self.remove_stopwords(tokens)
        return tokens

    def normalize(self, tokens):
        for i, token in enumerate(tokens):
            token = token.lower()
            token = self.ps.stem(token)
            tokens[i] = self.lem.lemmatize(token)
        return tokens
    
    def remove_links(self, text):
        text = re.sub('http[s]?://\S+', '', text)        
        return text

    def remove_punctuations(self, text):
        regular_expression = r'[^\w\s]'
        return re.sub(regular_expression, ' ', text.lower())

    def word_tokenize(self, text):
        tokenized_text = nltk.tokenize.word_tokenize(text)
        return tokenized_text

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

preprocessor = Preprocessor(stopwords=stopwords.words('english'))

In [4]:
data_path = "./data/phase-2/data.csv"
df = pd.read_csv(data_path).dropna()

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [9]:
def construct_positional_indexes(corpus: pd.DataFrame) -> dict:
    """
    Constructs positional indexes for the processed data by inserting words into a trie and creating positional indexes and posting lists.

    Parameters
    ----------
    corpus : dict
        The processed data.

    Returns
    -------
    dict
        The constructed positional indexes.
    """

    positional_index = {}

    for id, doc in corpus.iterrows():
        doc_id = doc['paperId']
        tokens = preprocessor.preprocess(doc['abstract'])
        for index, token in enumerate(tokens):
            if token not in positional_index:
                positional_index[token]['abstract'] = {'doc_count': 0, 'posting_list': {}}
            if doc_id not in positional_index[token]['posting_list']:
                positional_index[token]['abstract']['doc_count'] += 1
                positional_index[token]['abstract']['posting_list'][doc_id] = []
            positional_index[token]['abstract']['posting_list'][doc_id].append(index)
        tokens = preprocessor.preprocess(doc[''])
        for index, token in enumerate(tokens):
            if token not in positional_index:
                positional_index[token]['abstract'] = {'doc_count': 0, 'posting_list': {}}
            if doc_id not in positional_index[token]['posting_list']:
                positional_index[token]['abstract']['doc_count'] += 1
                positional_index[token]['abstract']['posting_list'][doc_id] = []
            positional_index[token]['abstract']['posting_list'][doc_id].append(index)
    return positional_index

index = construct_positional_indexes(df)

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [ ]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    # TODO: Implement the calculation of term frequency (TF) for the token in the specified document.
    pass

get_tf('deep', docs, '40ea606185b59cd07b456cb1022d64bf41f5538d')

In [ ]:
def get_idf(token: str) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    # TODO: Implement the calculation of IDF for the token based on your document collection.
    pass

get_idf('deep')

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [ ]:
def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    # TODO: Write the function to generate the list of dictionaries with TF-IDF scores.
    pass

tfidf = generate_tfidf_list(corpus, docs)

In [ ]:
def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    vector_list = [(term, weight) for term, weight in vector[doc_index].items() if weight > 0]
    vector_list.sort(key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in vector_list))
    normalized = {term: tfidf / length for term, tfidf in vector_list}
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

show_document_vector(tfidf, 0)

<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [ ]:
def search(query: str, max_result_count: int, method: str = 'ltc-lnc-tat', weight: float = 0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    pass

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [ ]:
def lm_search(query: str, max_result_count: int):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    pass

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [ ]:
class Evaluation:
    def __init__(self, actual, predicts):
        self.actual = actual
        self.queries = predicts

    def evaluate(self, require_scores):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        pass

    def precision(self):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        precision = 0

        return precision

    def recall(self):
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        recall = 0

        return recall

    def f1(self):
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results
        """
        f1_score = 0

        return f1_score

    def map(self):
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map_score = 0

        return map_score

    def ndcg(self):
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        ndcg_score = 0

        return ndcg_score

    def mrr(self):
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        mrr_score = 0

        return mrr_score

In [ ]:
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

# Read actuals and queries from validation.json file
actuals = []
queries = []

In [ ]:
predicts = [search(query, 10, method='ltc-lnc-tat') for query in queries]

eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)

In [ ]:
predicts = [search(query, 10, method='okapi25') for query in queries]

eval_bm = Evaluation(actuals, predicts)
eval_bm.evaluate(require_scores)

In [ ]:
predicts = [lm_search(query, 10) for query in queries]

eval_lm = Evaluation(actuals, predicts)
eval_lm.evaluate(require_scores)